# [Trapping Rain Water 1D](https://leetcode.com/problems/trapping-rain-water/)
- Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining.

<img src="./assets/4.png" width="650"/>

- Example
```
Input: [0,1,0,2,1,0,1,3,2,1,2,1]
Output: 6
```

## Solution 1
###### Observations
- Trap_water[i] hold by peak to the right and peak to the left
- Amount of water[i] = min(right_peak, left_peak) - height[i]
- Front and Rear cell(0, N-1) cant hold water

```C++
class Solution {
public:
    int trap(vector<int> &A) {
        int N = A.size();

        vector<int> max_from_left = vector<int>(N , 0);
        max_from_left[0] = A[0];
        for(int i=1; i<N; ++i) {
            max_from_left[i] = max(max_from_left[i-1], A[i]);
        }

        vector<int> max_from_right = vector<int>(N , 0);
        max_from_right[N-1] = A[N-1];
        for(int i=N-2; i>=0; --i) {
            max_from_right[i] = max(max_from_right[i+1], A[i]);
        }

        int ans = 0;
        for(int i=1; i<N-1; ++i) {
            int tmp = min(max_from_left[i-1], max_from_right[i+1]) - A[i];
            if(tmp > 0) ans += tmp;
        }
        return ans;
    }
};
```

## Solution 2
###### Observations
- Floodfill BFS
    - If water trapped at cell[i_next] $\to$ propagate with height = cell[i] (cell[i] is the wall)
    - If water leaked at cell[i_next] $\to$ propagate with new height = cell[i_next] (cell[i_next] becomes the new wall)

```C++
class Solution {
public:
    typedef pair<int,int> pii;
    vector<int> dir = {-1, 1};
    int trap(const vector<int> &A) {
        int N = A.size();

        // Minheap: [height] = index
        multimap<int, int> PQ;
        vector<bool> vis(N, false);

        // Add Front/rear wall
        PQ.insert( {A[0], 0} );
        PQ.insert( {A[N-1], N-1} );
        vis[0] = vis[N-1] = true;
        
        // BFS floodfill
        int ans = 0;
        while(!PQ.empty()) {
            auto [height_i, i] = *PQ.begin(); 
            PQ.erase(PQ.begin());

            // Propagate to next cell
            for(int i_: vector<int>{-1, 1}) {
                int i_next = i + i_;

                if(0 <= i_next && i_next < N &&
                        vis[i_next] == false) {
                    vis[i_next] = true;

                    // Case: water trapped at cell[i_next]; propagate with height_i
                    if(height_i > A[i_next]) {
                        PQ.insert( {height_i, i_next} );
                        ans += (height_i - A[i_next]);
                    }

                    // Case: water leaked at cell[i_next]; propagate with height_i_next
                    else PQ.insert( {A[i_next], i_next} );
                }
            }
        }
        return ans;
    }
};

```

## [Trapping Rain Water 2D](https://leetcode.com/problems/trapping-rain-water-ii/)

#### Solution
###### Obervations

```
12 13 1  12            12 13 1  12
13 4  13 12       ->   13 x  13 12
13 8  10 12            13 x  x  12
12 13 12 12            12 13 12 12
13 13 13 13            13 13 13 13
```
- Waters trapped by the cell 12 from the perimeter
- Use the floodfill bfs idea same as 1D

#### Code

```C++
class Solution {
public:
    int trapRainWater(vector<vector<int>> &A) {
        int N = A.size();
        int M = A[0].size();

        // Minheap: [height] = {x, y}
        multimap<int, pair<int,int>> PQ;
        vector<vector<bool>> vis(N, vector<bool>(M, false));

        // Add the perimeters
        for(int i=0; i<N; ++i) {
            PQ.insert( {A[i][0], {i, 0}} );
            vis[i][0] = true;

            PQ.insert( {A[i][M-1], {i, M-1}} );
            vis[i][M-1] = true;
        }
        for(int j=1; j<M-1; ++j) {
            PQ.insert( {A[0][j], {0, j}} );
            vis[0][j] = true;

            PQ.insert( {A[N-1][j], {N-1, j}} );
            vis[N-1][j] = true;
        }

        // floodfill bfs
        int ans = 0;
        while(!PQ.empty()) {
            auto [height_cur, pos] = *PQ.begin();
            auto [i, j] = pos;
            PQ.erase(PQ.begin());

            for(auto [i_, j_]: vector<pair<int,int>>{ {0,1}, {1, 0}, {0,-1}, {-1, 0} }) {
                int i_next = i + i_;
                int j_next = j + j_;

                if(0 <= i_next && i_next < N &&
                        0 <= j_next && j_next < M &&
                        vis[i_next][j_next] == false) {
                    vis[i_next][j_next] = true;

                    // Case water trapped
                    if(height_cur > A[i_next][j_next]) {
                        PQ.insert( {height_cur, {i_next, j_next}} );
                        ans += (height_cur - A[i_next][j_next]);
                    }

                    // Case water leaked, update new wall
                    else PQ.insert( {A[i_next][j_next], {i_next, j_next}} );
                }
            }
        }
        return ans;
    }
};
```